<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Global Neural Network Model
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook visualises the main results from the paper.
</p>

</div>


##  1. Loading Packages and Data

In this section, we import required libraries, define the model parameters and load the dataset.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
from models.global_model.model_functions.helper_functions.prepare_data import Prepare
from utils import create_pred_input
from scipy.interpolate import griddata
from models import MultivariateModelGlobal as Model       


os.environ['PYTHONHASHSEED'] = str(0)

#model parameters                    
lr = 0.001                      # Learning rate
min_delta = 1e-6               # Tolerance for optimization
patience = 50                   # Patience for early stopping
verbose = 2                     # Verbosity mode for optimization
n_countries=196
time_periods=63                 #

#prepare the data
data=pd.read_excel('../data/MainData.xlsx')
growth, precip, temp = Prepare(data, n_countries, time_periods)
x_train = {0:temp, 1:precip}

#summary statics for standardisation
mean_temp=np.nanmean(data["TempPopWeight"])
std_temp=np.nanstd(data["TempPopWeight"])
mean_precip=np.nanmean(data["PrecipPopWeight"])
std_precip=np.nanstd(data["PrecipPopWeight"])

pred_input, T, P= create_pred_input(mc=False, mean_temp=mean_temp, std_temp=std_temp, mean_precip=mean_precip, std_precip=std_precip)


#load results from simulations and retrieve best n models
n_models = 10  
date_of_run = '2025-06-05'
Model_selection='CV'

results = dict(np.load(f'../results/metrics/{Model_selection}/{date_of_run}/results.npy', 
                       allow_pickle=True).item())
results={k: v for k,v in results.items() if v is not None}
top_models = sorted(results, key=lambda node: results[node])[1:n_models]



2025-08-13 15:44:05.666367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 15:44:05.797267: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 15:44:05.808782: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 15:44:05.820152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755099845.835088   40422 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755099845.84

TypeError: create_pred_input() got an unexpected keyword argument 'mean_temp'

# 2. Data analysis

## 2.1 Comparing top-n models to Leirvik 

In [ ]:



# --- Build surfaces for each of the top-n models ---------------------
model_surfaces = []
for idx, node in enumerate(top_models, 1):
    
    # instantiate and load your model
    factory = Model(node, x_train, growth, dropout=0.2, penalty=0)
    factory.Depth=len(node)
    model=factory.get_model()
    weight_file = f'../results/Model Parameters/{Model_selection}/{date_of_run}/{node}.weights.h5'
    model.load_params(weight_file)

    # fit & predict
    model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)
   
    pred_flat = model.model_visual.predict(pred_input).reshape(-1,)
    Growth = pred_flat.reshape(T.shape) + 0.017

    opacity = 0.3
    surf = go.Surface(
        x=T, y=P, z=Growth,
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name=f'Model {node}'
    )
    model_surfaces.append(surf)
 



# --- Load benchmark data and create grid  ---------------------

# benchmark_data = pd.read_csv('../data/Benchmark/3d_results_Leirvik.csv')
# bench_temp   = benchmark_data['temp'].values
# bench_precip = benchmark_data['precip_value'].values * 1000
# bench_growth = benchmark_data['avg_prediction'].values

# bench_Z = griddata(
#     points=(bench_temp, bench_precip),
#     values=bench_growth,
#     xi=(T, P),
#     method='linear'
# )

# # --- Create the benchmark surface --------------------------------------
# bench_surface = go.Surface(
#     x=T, y=P, z=bench_Z,
#     colorscale='Reds',
#     showscale=False,
#     name='Benchmark'
# )





### 2.1.2 Data visualisation
In this section we visualise the top-n models in a temperature-precipitation grid 


In [ ]:
z=np.mean([surf.z for surf in model_surfaces], axis=0).reshape(T.shape)

mean_surface = go.Surface(
        x=T, y=P, z=z,
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name='mean_surface'
    )


mean_surface.y=mean_surface.y/1000

fig1 = go.Figure(data=mean_surface)
fig1.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',
        camera=dict(eye=dict(x=2.11, y=0.12, z=0.38)),
        zaxis=dict(range=[-0.15, 0.15])
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig1.show()








In [ ]:

fig.write_image("../results/images/Paper/3d_plot_model_vs_Leirvik.pdf", width=1600, height=1200, scale=2)



In [ ]:
# --- optionally: Save the figure as HTML ---------------------------------------------
fig.write_html(f'../results/images/10_best_average_vs_Leirvik.html')

## 2.2 Comparing Fixed Effects

In [ ]:
#define model of interest
node=(32, 16, 16)
factory = Model(node, x_train, growth, dropout=0.02, formulation=formulation, penalty=0.0)
factory.Depth=len(node)
model=factory.get_model()

weight_file = f'../results/Model Parameters/{Model_selection}/{date_of_run}/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

model.alpha


In [ ]:
#time fixed effects benchmark data
bench_time=pd.read_csv('../data/Benchmark/time_fixed_effects_Burke.csv')

# cleaning up the benchmark time data
bench_time['time'] = bench_time['Unnamed: 0'].astype(str).str.extract(r'(\d{4})')
bench_time['time'] = bench_time['time'].astype(float).astype('Int64')  # nullable integer
bench_time.drop(columns=['Unnamed: 0'], inplace=True)


#comparing model time fixed effects with benchmark time fixed effects
plt.figure(figsize=(10, 6))
plt.plot(bench_time.iloc[:, 1], bench_time.iloc[:, 0], label='Benchmark', color='red')
plt.plot(model.beta, label='Model', color='blue')




## 2.3 Making 2-D plots

In [ ]:



# --- load benchmark and model predictions as before ---------------------
benchmark_data = pd.read_csv('../data/Benchmark/3d_results_Leirvik.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values

# --- 1) find the 33rd percentile of precipitation ----------------------
percentile=90
# flatten the precip grid and compute 33rd percentile
p = np.percentile(np.array(P.flatten()), q=percentile)

# find the row index in P closest to that precip
# (assuming P is shape (n_precip, n_temp) from meshgrid)
idx = np.argmin(np.abs(P[:,0] - p))

# extract the corresponding temperature axis
temp_axis = T[idx, :]


# extract model and benchmark slices
model_slice = Growth[idx, :]
bench_slice = bench_Z[idx, :]

# --- 2) make the 2D line plot --------------------------------------------
fig2d = go.Figure([
    go.Scatter(
        x=temp_axis,
        y=model_slice,
        mode='lines+markers',
        name=f'Model @ P≈{p:.1f} mm',
    )])
# ,
#     go.Scatter(
#         x=temp_axis,
#         y=bench_slice,
#         mode='lines+markers',
#         name=f'Benchmark @ P≈{p:.1f} mm',
#     ),
# ])

fig2d.update_layout(
    title=f'Growth vs Temperature at {percentile}rd-percentile Precipitation ({p:.1f} mm)',
    xaxis_title='Temperature (°C)',
    yaxis_title='Growth',
    xaxis=dict(range=[0, 30]),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig2d.show()

# --- 3) (optional) save as html ------------------------------------------

# pio.write_html(fig2d, f'../results/images/2D_slice_Leirvik_comparison_{percentile}.html', auto_open=True)


## Appendix

In [ ]:



#visualise the model: 

print(model.alpha)

In [ ]:
np=model.beta.to_numpy()

last_siz=np[:58]
print('Last 6 sizes:', last_siz)

## 2d surfaces


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px  # for a built‑in qualitative palette

# 1) compute the 90th‑percentile precip level
percentile = 50
p = np.percentile(data['PrecipPopWeight'], percentile)

# 2) tolerance window = 0.1 % of p
tol = 0.005 * p

# 3) slice all obs around that precip band
obs_slice = data[np.abs(data['PrecipPopWeight'] - p) <= tol].copy()

# 4) (no need for top‑3 any more)

# 5) base model trace
fig2d = go.Figure([
    go.Scatter(
        x=temp_axis,
        y=model_slice,
        mode='lines+markers',
        name=f'Model @ P≈{p:.1f} mm',
        line=dict(width=2, color='black'),
        marker=dict(size=6)
    )
])

# 6) pick a color palette and assign one color per country
countries = obs_slice['CountryName'].unique()
palette   = px.colors.qualitative.Plotly  # 10 distinct colors
color_map = {c: palette[i % len(palette)] for i,c in enumerate(countries)}

# 7) plot all observations, colored by country
for country in countries:
    df_ctry = obs_slice[obs_slice['CountryName'] == country]
    fig2d.add_trace(
        go.Scatter(
            x=df_ctry['TempPopWeight'],
            y=df_ctry['GrowthWDI'],
            mode='markers',
            name=country,
            marker=dict(
                size=8,
                color=color_map[country],
                symbol='circle'
            )
        )
    )

# 8) finalize layout
fig2d.update_layout(
    title=f'Growth vs Temperature at {percentile}th‑percentile Precipitation ({p:.1f} mm)',
    xaxis_title='Temperature (°C)',
    yaxis_title='Growth',
    xaxis=dict(range=[0, 30]),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig2d.show()


## Model confidence plot - based on 10 best cv models

In [ ]:

results = dict(np.load(f'../results/metrics/15042025/results.npy', allow_pickle=True).item())

# Sort the keys (node configurations) by performance metric and select the top 10.
n_models = 10
date_of_run = '15042025'
ref_model = (8,2,2)  # Reference model configuration
top_models = sorted(results, key=lambda node: results[node])[:n_models]


# Initialize a list to store the prediction surfaces for each model.
prediction_surfaces = []
time_fixed_effects=[]
country_fixed_effects=[]

for node in top_models:
    
        # Instantiate your model with the given node configuration.
    model_instance = Model(nodes=node, x_train=x_train, y_train=growth, dropout=0, formulation="global")
    weight_file = f'../results/Model Parameters/CV/{date_of_run}/{str(node)}.weights.h5'
    model_instance.load_params(weight_file)  # Load the model weights
    model_instance.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=2)
    
    model_instance.in_sample_predictions
    
    # Use the model to predict on the standardized (T,P) grid.
    growth_pred_flat = model_instance.model_visual.predict(pred_input)
    growth_pred_flat = np.reshape(growth_pred_flat, (-1,))   # shape: (900,)
    pred = growth_pred_flat.reshape(T.shape)  # reshape to (30, 30)
    
    prediction_surfaces.append(pred)
    time_fixed_effects.append(model_instance.beta)
    country_fixed_effects.append(model_instance.alpha)


# Convert the list into a NumPy array: shape (10, 30, 30)
prediction_surfaces = np.array(prediction_surfaces)

# Calculate the pointwise mean and standard deviation across the 10 models.
ensemble_mean = np.mean(prediction_surfaces, axis=0)
ensemble_std = np.std(prediction_surfaces, axis=0)

# Select the Chosen Model (e.g. configuration (8,2,2))
chosen_index = top_models.index(ref_model)
chosen_surface = prediction_surfaces[chosen_index]

model_confidence_plot(ref_model, chosen_surface, ensemble_std=ensemble_std, T=T, P=P, save_as_html=False)



## plotting the fixed effects from the model

In [ ]:

# Squeeze to shape (10, num_countries)
country_FE = np.array(country_fixed_effects).squeeze(axis=1)

# Number of models
n_models = country_FE.shape[0]

# Compute mean & sample std across the 10 models (result shape: num_countries)
country_FE_mean = np.mean(country_FE, axis=0).squeeze()
country_FE_std  = np.std(country_FE,  axis=0, ddof=1).squeeze()

# Standard error and 95% CI
se    = country_FE_std / np.sqrt(n_models)
t_val = stats.t.ppf(0.975, df=n_models - 1)
ci    = t_val * se


# X-axis indices 
country_indices = np.arange(country_FE_mean.shape[0])

plt.figure(figsize=(12, 6))
plt.plot(x, country_FE_mean, label='Mean')

plt.fill_between(
    country_indices,
    country_FE_mean - ci,
    country_FE_mean + ci,
    alpha=0.3,
    label='95% CI'
)
plt.title('Mean and 95% Confidence Interval of Country Fixed Effects')
plt.xlabel('Country Index')
plt.ylabel('Country Fixed Effect')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
